## styletransfer

source code for OpenSea collection: [Style Transfer Arts](https://opensea.io/collection/styletransfer)

### 1. import libary

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from PIL import Image

### 2. Configuration

In [ ]:
max_dim = 640
content_path = '1.jpg'
style_path = '2.jpg'
stpes = 501
style_weight=5.0
content_weight=0.025
total_variation_weight=30
save_img = True

In [ ]:
def load_img(path_to_img):
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, 
                                channels=3, 
                                dtype=tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [ ]:
def imshow(image, title=None):    
    if len(image.shape) > 3:
        image = tf.squeeze(image)
    plt.imshow(image)
    
    if title:
        plt.title(title)
        
    plt.axis('off')
    plt.show()

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

### 3. load VGG16 Pre-train model

In [ ]:
vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

### 4. nn building

In [ ]:
content_layers = ['block4_conv2'] 

# content_layers = ['block2_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

In [ ]:
def vgg_model(layer_names):
    vgg = tf.keras.applications.VGG16(include_top=False, \
    weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

In [ ]:
def gram_matrix(input_tensor):

    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

In [ ]:
class StyleContentModel(tf.keras.models.Model):
    
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg =  vgg_model(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)

    def call(self, inputs):
        
        preprocessed_input = tf.keras.applications.vgg16.preprocess_input(inputs*255.0)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                          outputs[self.num_style_layers:])
        style_outputs = [gram_matrix(style_output) for style_output in style_outputs]
        style_dict = {style_name:value for style_name, 
                      value in zip(self.style_layers, style_outputs)}
        
        content_dict = {content_name:value for content_name, 
                        value in zip(self.content_layers, content_outputs)}
        
        return {'content':content_dict, 'style':style_dict}

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)
results = extractor(tf.constant(content_image))
style_results = results['style']

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']
image = tf.Variable(content_image)

opt = tf.optimizers.Adam(learning_rate=0.02, 
                         beta_1=0.99, 
                         epsilon=1e-1)

def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

In [ ]:
def style_content_loss(outputs):    
    style_outputs = outputs['style']    
    content_outputs = outputs['content']    
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-\
                                           style_targets[name])**2) 
                           for name in style_outputs.keys()])    
    style_loss *= style_weight / num_style_layers   
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-\
                                             content_targets[name])**2) 
                             for name in content_outputs.keys()])
    
    content_loss *= content_weight / num_content_layers    
    loss = style_loss + content_loss
    return loss

def total_variation_loss(image):
    x_deltas = image[:,:,1:,:] - image[:,:,:-1,:]
    y_deltas = image[:,1:,:,:] - image[:,:-1,:,:]
    return tf.reduce_mean(x_deltas**2) + tf.reduce_mean(y_deltas**2)

In [ ]:
@tf.function()

def train_step(image):

    with tf.GradientTape() as tape:
        
        outputs = extractor(image)
        loss = style_content_loss(outputs)
        loss += total_variation_weight*total_variation_loss(image)
    
    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

### 5. model training

In [ ]:
for n in range(stpes):
    train_step(image)
    if n%50==0:
        imshow(image.read_value(), "epoch: {}".format(n))
        if save_img==True:
            s_image = tf.squeeze(image)
            s_image = Image.fromarray(np.uint8(s_image.numpy()*255))
            s_image.save('result/'+'steps_'+str(n)+'.jpg')